In [2]:
from sedfitter.sed import SEDCube
import sedfitter
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as const
import astropy.units as u
import os

In [7]:
models = os.listdir('../data/galaxySEDs/')
if '.DS_Store' in models: models.remove('.DS_Store')
models = [x for x in models if 'norm' not in x]
print(len(models))

32


In [ ]:
#Testing first SESNA data point with fitting algorithm

